In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.unlock())

import time
import cv2
from l42 import *
from local_nav import *
from VisGraph import *
from update_pos import *
from kalman_filter import *


In [2]:
#getters and setters functions

@tdmclient.notebook.sync_var
def get_prox_value():
    global prox_horizontal
    return prox_horizontal

#@tdmclient.notebook.sync_var
#def get_ground_value():
#    global prox_ground_reflected
#    return prox_ground_reflected

@tdmclient.notebook.sync_var
def get_left_speed():
    global motor_left_target
    return motor_left_target

@tdmclient.notebook.sync_var
def get_right_speed():
    global motor_right_target
    return motor_right_target

@tdmclient.notebook.sync_var
def set_motor_speed(left_speed, right_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

In [3]:
VideoCap = cv2.VideoCapture(0)
ret, frame = VideoCap.read()
#Create constant variables
next_goal_trigger = 20
delta_t = 0.05
start = []
stop = []
#Creation of the map and the global path
while(len(stop)==0 or (len(start)==0)):
    ret, frame_init = VideoCap.read()
    start, stop = detect_start_stop(frame_init)
    
global_path = initialisation(frame, start, stop)
start = global_path[0]
stop = global_path[len(global_path)-1]
    
#Set the initial motor speeds at 0
set_motor_speed(0, 0)

#Initialisation of the variables
prox_sensors = np.array([0, 0, 0, 0, 0, 0, 0])

#Initialisation Kalman filter
kalman_filter = KalmanFilter(delta_t)

#Find the initial position and orientation of the robot
position, orientation = detect_thymio(start, 0, 0, 0, kalman_filter, frame)

#Initialisation of the variables that will keep in memory the position and orientation of the last iteration
old_position = position
old_orientation = orientation


#Initialisation of the variables that will keep track of the current goal position and orientation
current_goal = 1
current_goal_pos = global_path[current_goal]

distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

#Initialisation of the variables that will be updated to set the speed of the wheels
left_speed = 0
right_speed = 0

#Initialisation of the boolean that will recall if the camera detection is possible or not at each iteration
possible_detection = True

while(True):
    ret, frame = VideoCap.read()
    current_goal_pos = global_path[current_goal]
    #Recall the values of the sensors and the wheel speeds 
    prox_sensors = get_prox_value()
    #prox_ground_values = get_ground_value()
    #set_motor_speed(100, 100)
    
    #Reset and update the detection boolean
    #possible_detection = #fonction prévue à cet effet
    
    #if possible_detection :
    #Vision : Returns the position and the orientation of the robot from the camera
        
    position, orientation = detect_thymio(old_position, old_orientation, left_speed, right_speed, kalman_filter, frame)
    cv2.circle(frame, (int(position[0]), int(position[1])), 10, (255, 0, 0), 2)
    x2 = position[0] + 50 * np.cos(orientation)
    y2 = position[1] + 50 * np.sin(orientation)
    cv2.line(frame, (int(position[0]), int(position[1])), (int(x2), int(y2)),(0, 255, 0), 2)
    for i in range (len(global_path)-1):
        cv2.line(frame, (int(global_path[i][0]), int(global_path[i][1])), (int(global_path[i+1][0]), int(global_path[i+1][1])),(0, 0, 255), 2)
    cv2.imshow('image', frame)

    #else :
        #The position and orientation of the robot is estimated from its last state and the wheel speeds
        #position = kalman(position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction 
        #orientation = kalman(old_position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction
    
    #Update the current goal orientation and the distance from the robot
    distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)
    print("left speed = ")
    print(left_speed)
    print("right speed = ")
    print(right_speed)
    #Find the speeds to apply to the robot
    left_speed, right_speed = localNavigation(orientation, old_orientation, current_goal_orientation, prox_sensors)
    #Apply the desired speeds to the robot
    set_motor_speed(int(left_speed), int(right_speed))
    
    #Keep in memory the position and orientation of the robot for the next 
    old_position = position
    old_orientation = orientation
    
    #Update the current goal if necessary
    if (distance_to_goal < next_goal_trigger) :
        current_goal += 1
    
    if (current_goal == len(global_path)) :
        set_motor_speed(0, 0)
        break
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
    await client.sleep(0.05)

cap.release()
cv2.destroyAllWindows()



bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 221.95it/s]


No thymio
No thymio
Predict
0.04
Position prédite = 
[90.07, 50.07]
Angle prédit = 
0.04
No thymio
Update
left speed = 
0
right speed = 
0
No thymio
Update
left speed = 
176.39149587375385
right speed = 
123.60850412624615
No thymio
Update
left speed = 
174.86541411432157
right speed = 
125.13458588567843
No thymio
Update
left speed = 
175.10250788768406
right speed = 
124.89749211231593
No thymio
Update
left speed = 
178.86258114052316
right speed = 
121.13741885947685
No thymio
Update
left speed = 
174.02924607223952
right speed = 
125.97075392776048
No thymio
Update
left speed = 
179.1902513092535
right speed = 
120.8097486907465
No thymio
Update
left speed = 
171.22828029734956
right speed = 
128.77171970265044
No thymio
Update
left speed = 
173.08644107600264
right speed = 
126.91355892399736
No thymio
Update
left speed = 
173.50743546119872
right speed = 
126.49256453880128
No thymio
Update
left speed = 
173.18644107600264
right speed = 
126.81355892399736
No thymio
Update
left s

No thymio
Predict
0.6027321112467142
Position prédite = 
[175.68314349844502, 52.24404215051322]
Angle prédit = 
0.6027321112467142
left speed = 
110.75027143607022
right speed = 
189.24972856392978
No thymio
No thymio
Predict
0.6012441337941047
Position prédite = 
[176.87649332178336, 53.08708115770311]
Angle prédit = 
0.6012441337941047
left speed = 
110.38189985719251
right speed = 
189.6181001428075
No thymio
No thymio
Predict
0.5993038566772165
Position prédite = 
[178.0709921657187, 53.928447790517545]
Angle prédit = 
0.5993038566772165
left speed = 
109.94998549322001
right speed = 
190.05001450677997
No thymio
No thymio
Predict
0.5969621287291972
Position prédite = 
[179.2669855570555, 54.767631133354115]
Angle prédit = 
0.5969621287291972
left speed = 
109.56662818812276
right speed = 
190.43337181187724
No thymio
No thymio
Predict
0.5943630470992173
Position prédite = 
[180.46477707757933, 55.60417559942632]
Angle prédit = 
0.5943630470992173
left speed = 
109.32087352448124


No thymio
Predict
0.3294245400644408
Position prédite = 
[228.721851868823, 82.56536151510058]
Angle prédit = 
0.3294245400644408
left speed = 
101.29036673404923
right speed = 
198.70963326595077
No thymio
No thymio
Predict
0.3181202129509908
Position prédite = 
[230.08216367880766, 83.07649592233436]
Angle prédit = 
0.3181202129509908
left speed = 
101.00797642734531
right speed = 
198.9920235726547
No thymio
No thymio
Predict
0.3065490794141668
Position prédite = 
[231.44737966804297, 83.5730163479421]
Angle prédit = 
0.3065490794141668
left speed = 
100.75319506057347
right speed = 
199.2468049394265
No thymio
No thymio
Predict
0.2946328457726409
Position prédite = 
[232.81744416394758, 84.05452143748491]
Angle prédit = 
0.2946328457726409
left speed = 
100.42364873542442
right speed = 
199.57635126457558
No thymio
No thymio
Predict
0.2824117502724375
Position prédite = 
[234.1923198333032, 84.52050580555405]
Angle prédit = 
0.2824117502724375
left speed = 
100.13252710481217
right

...

No thymio
Predict
0.12404234740156517
Position prédite = 
[382.22795126944, 223.69310898462913]
Angle prédit = 
0.12404234740156517
left speed = 
105.03610435743403
right speed = 
194.96389564256597
No thymio
No thymio
Predict
0.11686617032268004
Position prédite = 
[383.65111027942453, 223.9318242060202]
Angle prédit = 
0.11686617032268004
left speed = 
104.95006933030115
right speed = 
195.04993066969885
No thymio
No thymio
Predict
0.10960844010746959
Position prédite = 
[385.0754451673067, 224.16082465789512]
Angle prédit = 
0.10960844010746959
left speed = 
104.87219182167621
right speed = 
195.1278081783238
No thymio
No thymio
Predict
0.10217252993927636
Position prédite = 
[386.50089835804175, 224.37999161119785]
Angle prédit = 
0.10217252993927636
left speed = 
104.70204250001368
right speed = 
195.29795749998632
No thymio
No thymio
Predict
0.09468687790255254
Position prédite = 
[387.9274232574186, 224.58907550531242]
Angle prédit = 
0.09468687790255254
left speed = 
104.654542

No thymio
No thymio
Predict
-0.30272596361692916
Position prédite = 
[439.10726640564695, 223.60418940620954]
Angle prédit = 
-0.30272596361692916
left speed = 
95.91685885997853
right speed = 
204.08314114002147
No thymio
No thymio
Predict
-0.3198983600350969
Position prédite = 
[440.4788946283285, 223.26765760899028]
Angle prédit = 
-0.3198983600350969
left speed = 
95.4043830098354
right speed = 
204.59561699016461
No thymio
No thymio
Predict
-0.33765292204047725
Position prédite = 
[441.84335015430133, 222.9088347743491]
Angle prédit = 
-0.33765292204047725
left speed = 
94.84845582441812
right speed = 
205.15154417558188
No thymio
No thymio
Predict
-0.35613574241952295
Position prédite = 
[443.19998850198726, 222.52709824218076]
Angle prédit = 
-0.35613574241952295
left speed = 
94.1530203043167
right speed = 
205.8469796956833
No thymio
No thymio
Predict
-0.3752629087332686
Position prédite = 
[444.5480581988623, 222.12165951620105]
Angle prédit = 
-0.3752629087332686
left speed 

No thymio
Predict
0.9986878001670791
Position prédite = 
[488.65430971785094, 228.35373546002748]
Angle prédit = 
0.9986878001670791
left speed = 
127.65319063532667
right speed = 
172.34680936467333
No thymio
No thymio
Predict
1.0133469767903813
Position prédite = 
[489.4625906521313, 229.57022810907043]
Angle prédit = 
1.0133469767903813
left speed = 
125.80129618547925
right speed = 
174.19870381452074
No thymio
No thymio
Predict
1.0261537104825822
Position prédite = 
[490.25398622638625, 230.7974197775838]
Angle prédit = 
1.0261537104825822
left speed = 
124.03234348979676
right speed = 
175.96765651020323
No thymio
No thymio
Predict
1.0372974408232334
Position prédite = 
[491.0305032021036, 232.03375501895076]
Angle prédit = 
1.0372974408232334
left speed = 
122.44429226713174
right speed = 
177.55570773286826
No thymio
No thymio
Predict
1.0469423980828823
Position prédite = 
[491.79397925326197, 233.27789091342856]
Angle prédit = 
1.0469423980828823
left speed = 
121.013069400644

Update
left speed = 
117.94060910193235
right speed = 
182.05939089806765
No thymio
Update
left speed = 
113.65779055289337
right speed = 
186.34220944710663
No thymio
Update
left speed = 
115.37592136048511
right speed = 
184.6240786395149
No thymio
Update
left speed = 
119.56499236238997
right speed = 
180.43500763761003
No thymio
Update
left speed = 
122.09978208346463
right speed = 
177.90021791653538
No thymio
Update
left speed = 
114.25854439065847
right speed = 
185.74145560934153
No thymio
Update
left speed = 
114.97300533488385
right speed = 
185.02699466511615
No thymio
Update
left speed = 
115.54028145025836
right speed = 
184.45971854974164
No thymio
Update
left speed = 
114.21080078777146
right speed = 
185.78919921222854
No thymio
Update
left speed = 
114.5206276752671
right speed = 
185.4793723247329
No thymio
Update
left speed = 
116.98263715274742
right speed = 
183.01736284725257
No thymio
Update
left speed = 
113.83329118277365
right speed = 
186.16670881722635
No th

NameError: name 'cap' is not defined

In [ ]:
await tdmclient.notebook.stop()